# Sistema ML: Detección de **Riesgo de Corrupción** en Obras Públicas (Perú)

Este notebook implementa un *pipeline* end-to-end para detectar obras públicas con **riesgo de corrupción**:
1) **Ingesta** de datos (simulada si no se detectan archivos reales).  
2) **ETL** y *feature engineering* con banderas de riesgo.  
3) Entrenamiento de **modelos** (*baseline* y árbol de decisión/ensamble).  
4) **Evaluación** con métricas y curvas.  
5) **XAI**: Importancia por permutación y **PDP/ICE**.  
6) **Exportación** de artefactos (pipeline + modelo) y función de inferencia.

> ⚠️ Reemplace los *placeholders* de rutas por sus fuentes reales (SIAF, SEACE/OSCE, INFObras, Módulos CGR/BID, etc.) cuando estén disponibles.


## Contexto y objetivo

En el marco de la **Contraloría (CGR)** y el proyecto **BID‑3**, buscamos **priorizar** y **alertar** sobre obras con probabilidad de incurrir en **riesgos de corrupción** (p. ej., adicionales y ampliaciones atípicas, fraccionamiento, sanciones previas de empresas, colusión, sobrecostos, baja competencia, etc.).

**Variable objetivo (label)**: `riesgo_corrupcion` (1 = alto riesgo, 0 = bajo riesgo).  
**Fuentes típicas:** SIAF, SEACE/OSCE, INFOBRAS, PERUCOMPRAS, SERVIR, SNCP, padrones de sanciones, y registros internos de la CGR.


In [1]:
# == 0. Setup
import os
import sys
import math
import json
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from typing import List, Tuple

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (classification_report, confusion_matrix, roc_auc_score, average_precision_score,
                             RocCurveDisplay, PrecisionRecallDisplay, ConfusionMatrixDisplay, brier_score_loss)
from sklearn.inspection import permutation_importance, PartialDependenceDisplay

# Ajustes visuales (sin estilos/colores específicos)
plt.rcParams['figure.figsize'] = (7, 5)

ARTIF_DIR = Path('artifacts')
SCRIPT_DIR = Path('scripts')
ARTIF_DIR.mkdir(exist_ok=True, parents=True)
SCRIPT_DIR.mkdir(exist_ok=True, parents=True)

print('Versions:')
import sklearn, matplotlib
print('sklearn:', sklearn.__version__)
print('pandas :', pd.__version__)
print('numpy  :', np.__version__)
print('matplot:', matplotlib.__version__)


Versions:
sklearn: 1.7.2
pandas : 2.3.3
numpy  : 2.3.3
matplot: 3.10.7


In [4]:
# == 0.1 Intento de lectura de diccionarios (si existen)
dict_files = [
    '../data/external/catalogos/Diccionario_Datos_ML_Completo_V1.xlsx',
    '../data/external/catalogos/Diccionario_Datos_Sistemas_Fuente_V1.xlsx'
]

loaded_dicts = {}
for f in dict_files:
    if os.path.exists(f):
        try:
            df = pd.read_excel(f)
            loaded_dicts[os.path.basename(f)] = df.head(10)
        except Exception as e:
            print(f'[WARN] No se pudo leer {f}:', e)

if loaded_dicts:
    print('Se cargaron vistas previas de los diccionarios:')
    for name, df in loaded_dicts.items():
        print(' -', name, '=> shape:', df.shape)
else:
    print('No se encontraron/leyeron diccionarios en /data por ahora.')


Se cargaron vistas previas de los diccionarios:
 - Diccionario_Datos_ML_Completo_V1.xlsx => shape: (10, 7)
 - Diccionario_Datos_Sistemas_Fuente_V1.xlsx => shape: (10, 5)


## Modelo de datos (referencial)

A continuación, diagramas referenciales del **Data Warehouse** para Obras, Empresas y Miembros de comité/equipo. Úselos como guía de *staging → silver → gold* y llaves de integración (CUI, RUC, N° contrato, etc.).

![Matriz ML DW](/data/processed/Matriz_ML_DW.png)

![Obras ML DW](/data/processed/Obras_ML_DW.png)

![Empresa ML DW](/data/processed/Empresa_ML_DW.png)

![Miembro ML DW](/data/processed/Miembro_ML_DW.png)


In [7]:
from pathlib import Path
import pandas as pd, numpy as np, re

P_OBRA = DATA_ROOT / "obra"
P_EMP  = DATA_ROOT / "empresa"
P_FUNC = DATA_ROOT / "funcionario"

def read_any(path: Path) -> pd.DataFrame:
    if path.suffix.lower() in [".xlsx", ".xls"]:
        # lee primera hoja; si necesitas otra, usa sheet_name="..."
        return pd.read_excel(path)
    for enc in ["utf-8-sig","latin-1"]:
        try: return pd.read_csv(path, encoding=enc)
        except Exception: pass
    return pd.read_csv(path)

def concat_many(folder: Path, patterns) -> pd.DataFrame:
    files = []
    for pat in patterns:
        files += list(folder.glob(pat))
    files = sorted(set(files))
    if not files: return pd.DataFrame()
    dfs = []
    for f in files:
        try:
            df = read_any(f)
            df["_source"] = f.name
            dfs.append(df)
        except Exception as e:
            print(f"[WARN] {f.name}: {e}")
    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

obra_raw = concat_many(P_OBRA, patterns=[
    "DS_DASH_Obra_*.*",               # tus CSVs
    "Datos generales*Obra*.xlsx"      # tus excels “Datos generales …”
])
emp_raw  = concat_many(P_EMP, patterns=[
    "DS_DASH_Empresa_*.*",
    "Datos generales*Empresa*.xlsx"
])
func_raw = concat_many(P_FUNC, patterns=[
    "DS_DASH_Miembro_*.*",
    "Datos generales*Funcion*.*"
])

print("[Obra] shape:", obra_raw.shape, " [Empresa] shape:", emp_raw.shape, " [Func] shape:", func_raw.shape)
assert not obra_raw.empty, "No se cargó nada de 'obra' — revisa DATA_ROOT y patrones."

# --- Estandariza columnas a lo que pide el modelo ---
colmap_obras = {
    "costo_total": ["MontoContrato","CostoTotal","costo_total","Monto total","Monto_total"],
    "plazo_meses": ["PlazoMeses","plazo_meses","Plazo (meses)"],
    "adicionales_pct": ["AdicPct","%Adicionales","adicionales_pct","Porc_Adicionales"],
    "ampliaciones": ["NroAmpliaciones","ampliaciones"],
    "penalidades": ["NroPenalidades","penalidades"],
    "baja_competencia": ["BajaCompetencia","baja_competencia","PocosPostores"],
    "consorcio": ["Consorcio","consorcio"],
    "experiencia_entidad": ["ExperienciaEntidad","experiencia_entidad"],
    "region_riesgo": ["RegionRiesgo","region_riesgo"],
    "tipo_proceso": ["TipoProceso","tipo_proceso"],
    "RUC": ["RUC","ruc"],
    "CUI": ["CUI","cui"],
    "riesgo_corrupcion": ["riesgo_corrupcion"]
}
colmap_empresa = {
    "RUC": ["RUC","ruc"],
    "empresa_sancionada": ["EmpresaSancionada","SancionOSCE","empresa_sancionada"]
}

def standardize(df, colmap):
    if df.empty: return df
    out = df.copy()
    for std, variants in colmap.items():
        for v in variants:
            if v in out.columns:
                out[std] = out[v]
                break
        if std not in out.columns:
            out[std] = np.nan
    return out

obra = standardize(obra_raw, colmap_obras)
emp  = standardize(emp_raw,  colmap_empresa)

# --- Limpieza numéricos/porcentajes ---
def to_num(x):
    if pd.isna(x): return np.nan
    if isinstance(x,str):
        x = x.replace("S/.","").replace(",","").strip()
        x = re.sub(r"[^0-9\.\-eE]", "", x)
    try: return float(x)
    except: return np.nan

for c in ["costo_total","plazo_meses","adicionales_pct","ampliaciones","penalidades",
          "baja_competencia","consorcio","experiencia_entidad"]:
    if c in obra.columns: obra[c] = obra[c].map(to_num)

if "adicionales_pct" in obra.columns:
    obra.loc[obra["adicionales_pct"]>1, "adicionales_pct"] = obra["adicionales_pct"]/100.0

def norm_tipo_proceso(s):
    if pd.isna(s): return np.nan
    s = str(s).lower()
    if "directa" in s: return "Contratación Directa"
    if "simplificada" in s: return "Adjudicación Simplificada"
    return "Licitación"
if "tipo_proceso" in obra.columns:
    obra["tipo_proceso"] = obra["tipo_proceso"].apply(norm_tipo_proceso)

def norm_region(s):
    if pd.isna(s): return "MEDIA"
    s = str(s).upper()
    return s if s in {"ALTA","MEDIA","BAJA"} else "MEDIA"
obra["region_riesgo"] = obra.get("region_riesgo","MEDIA")
obra["region_riesgo"] = obra["region_riesgo"].apply(norm_region)

# empresa_sancionada por RUC
obra["empresa_sancionada"] = obra.get("empresa_sancionada", np.nan)
if not emp.empty and "RUC" in obra.columns and "RUC" in emp.columns:
    aux = emp[["RUC","empresa_sancionada"]].copy()
    if aux["empresa_sancionada"].dtype == object:
        aux["empresa_sancionada"] = aux["empresa_sancionada"].astype(str).str.lower().map(
            {"si":1,"sí":1,"true":1,"1":1,"no":0,"false":0,"0":0}
        ).fillna(0)
    obra = obra.merge(aux, on="RUC", how="left", suffixes=("","_emp"))
    obra["empresa_sancionada"] = obra["empresa_sancionada"].fillna(obra["empresa_sancionada_emp"])
    obra.drop(columns=[c for c in obra.columns if c.endswith("_emp")], inplace=True)
obra["empresa_sancionada"] = obra["empresa_sancionada"].fillna(0).astype(int)

# --- Dataset final
features_needed = [
    "costo_total","plazo_meses","adicionales_pct","ampliaciones","penalidades",
    "baja_competencia","empresa_sancionada","consorcio","experiencia_entidad",
    "region_riesgo","tipo_proceso"
]
label_col = "riesgo_corrupcion"

for c in features_needed:
    if c not in obra.columns: obra[c] = np.nan

if label_col in obra.columns:
    obra[label_col] = pd.to_numeric(obra[label_col], errors="coerce").fillna(obra[label_col]).astype(int)

df = obra[features_needed + ([label_col] if label_col in obra.columns else [])].copy()
print("Dataset ML listo. shape:", df.shape)
df.head(3)


[Obra] shape: (0, 0)  [Empresa] shape: (0, 0)  [Func] shape: (0, 0)


AssertionError: No se cargó nada de 'obra' — revisa DATA_ROOT y patrones.

In [ ]:
# == 2. ETL y Feature Engineering
# Limpiezas básicas y creación de *flags* (ejemplos). Ajuste según su diccionario.
raw = df.copy()

# Ejemplo de winsorización simple para variables monetarias (evitar outliers extremos)
def winsorize(s: pd.Series, p_low=0.01, p_high=0.99) -> pd.Series:
    lo, hi = s.quantile(p_low), s.quantile(p_high)
    return s.clip(lo, hi)

num_cols = ['costo_total', 'plazo_meses', 'adicionales_pct', 'ampliaciones', 'penalidades']
for c in num_cols:
    raw[c] = winsorize(raw[c])

# Banderas compuestas (red flags)
raw['flag_adicionales_altos'] = (raw['adicionales_pct'] > 0.15).astype(int)
raw['flag_muchas_ampliaciones'] = (raw['ampliaciones'] >= 2).astype(int)
raw['flag_penalidades'] = (raw['penalidades'] >= 1).astype(int)
raw['flag_contratacion_directa'] = (raw['tipo_proceso'] == 'Contratación Directa').astype(int)
raw['flag_region_alta'] = (raw['region_riesgo'] == 'ALTA').astype(int)

# Definimos features/target
target = 'riesgo_corrupcion'
features = [c for c in raw.columns if c != target]

X = raw[features].copy()
y = raw[target].astype(int)

# Columnas por tipo
cat_cols = X.select_dtypes(include=['object']).columns.tolist()
bin_cols = [c for c in X.columns if X[c].dropna().nunique() == 2 and X[c].dtype != 'object']
num_cols = [c for c in X.columns if c not in cat_cols + bin_cols]

print('Num cols:', num_cols)
print('Bin cols:', bin_cols)
print('Cat cols:', cat_cols)

# Pipelines
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('ohe', OneHotEncoder(handle_unknown='ignore'))
])

preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols),
        ('pass', 'passthrough', bin_cols)  # flags ya binarios
    ]
)

# Splits
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)

print('[Split] Train:', X_train.shape, ' Test:', X_test.shape)


In [ ]:
# == 3. Entrenamiento & Evaluación

def evaluar_modelo(modelo: Pipeline, X_tr, y_tr, X_te, y_te, nombre='modelo'):
    modelo.fit(X_tr, y_tr)
    y_pred = modelo.predict(X_te)
    if hasattr(modelo, 'predict_proba'):
        y_proba = modelo.predict_proba(X_te)[:, 1]
    else:
        # algunos modelos (p. ej. SVM linear sin probas); fallback con decisión si existe
        y_proba = getattr(modelo, 'decision_function', lambda x: y_pred)(X_te)
        # reescalado simple si fuera necesario
        if y_proba.ndim == 1 and (y_proba.max() > 1 or y_proba.min() < 0):
            from sklearn.preprocessing import MinMaxScaler
            y_proba = MinMaxScaler().fit_transform(y_proba.reshape(-1,1)).ravel()

    roc = roc_auc_score(y_te, y_proba)
    pr  = average_precision_score(y_te, y_proba)

    print(f'== {nombre} ==')
    print('ROC-AUC :', round(roc, 4))
    print('PR-AUC  :', round(pr, 4))
    print('\nClassification Report:\n', classification_report(y_te, y_pred, digits=4))

    fig1, ax1 = plt.subplots()
    RocCurveDisplay.from_predictions(y_te, y_proba, ax=ax1)
    ax1.set_title(f'ROC — {nombre}')
    plt.show()

    fig2, ax2 = plt.subplots()
    PrecisionRecallDisplay.from_predictions(y_te, y_proba, ax=ax2)
    ax2.set_title(f'Precision-Recall — {nombre}')
    plt.show()

    fig3, ax3 = plt.subplots()
    ConfusionMatrixDisplay.from_predictions(y_te, y_pred, ax=ax3)
    ax3.set_title(f'Matriz de confusión — {nombre}')
    plt.show()

    # Brier score (calibración)
    try:
        brier = brier_score_loss(y_te, y_proba)
        print('Brier score:', round(brier, 4))
    except Exception:
        pass

    return {'roc_auc': roc, 'pr_auc': pr, 'model': modelo}

# Baseline: Regresión Logística con balanceo
baseline = Pipeline(steps=[
    ('prep', preprocess),
    ('clf', LogisticRegression(max_iter=200, class_weight='balanced', n_jobs=None, solver='liblinear'))
])

res_base = evaluar_modelo(baseline, X_train, y_train, X_test, y_test, nombre='Baseline — RegLog')


In [ ]:
# RandomForest con pequeña búsqueda de hiperparámetros (rápida)
rf_pipe = Pipeline(steps=[
    ('prep', preprocess),
    ('rf', RandomForestClassifier(class_weight='balanced', random_state=42))
])

param_grid = {
    'rf__n_estimators': [150, 300],
    'rf__max_depth': [None, 8, 12],
    'rf__min_samples_split': [2, 10],
    'rf__min_samples_leaf': [1, 3]
}

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
gs = GridSearchCV(rf_pipe, param_grid, scoring='average_precision', cv=cv, n_jobs=-1, verbose=1)
gs.fit(X_train, y_train)

print('Mejores params:', gs.best_params_)
print('Mejor score (PR-AUC cv):', round(gs.best_score_, 4))

best_rf = gs.best_estimator_
res_rf = evaluar_modelo(best_rf, X_train, y_train, X_test, y_test, nombre='RandomForest (mejor)')


In [ ]:
# Selección simple del mejor modelo por PR-AUC (priorizamos alertas en clase positiva)
best = res_rf if res_rf['pr_auc'] >= res_base['pr_auc'] else res_base
best_name = 'RandomForest' if best is res_rf else 'RegLog'
print('Modelo seleccionado:', best_name, '— PR-AUC:', round(best['pr_auc'], 4), ' ROC-AUC:', round(best['roc_auc'], 4))

best_model = best['model']


In [ ]:
# == 4. XAI (explicabilidad)
# 4.1 Importancia por permutación (global)
try:
    pi_result = permutation_importance(best_model, X_test, y_test, n_repeats=5, random_state=42, n_jobs=-1)
    # Recuperar nombres de features post-transformación
    # ColumnTransformer + OneHot => necesitamos nombres expandidos
    def get_feature_names(column_transformer, input_features):
        # Basado en scikit-learn >=1.0
        output = []
        for name, trans, cols in column_transformer.transformers_:
            if name == 'remainder' and trans == 'drop':
                continue
            if name == 'pass' and trans == 'passthrough':
                # columnas pasadas tal cual
                if cols is None:
                    cols = [f for f in input_features if f not in sum([list(ct[2]) for ct in column_transformer.transformers_ if ct[0]!=name], [])]
                output.extend([input_features[i] if isinstance(i, int) else i for i in cols])
            else:
                if hasattr(trans, 'get_feature_names_out'):
                    # ej: 'num' StandardScaler -> no agrega sufijos; 'cat' OHE -> expande
                    feat_names = trans.get_feature_names_out(cols)
                    output.extend(feat_names.tolist())
                elif trans == 'passthrough':
                    output.extend([input_features[i] if isinstance(i, int) else i for i in cols])
                else:
                    # fallback
                    output.extend([str(c) for c in cols])
        return output

    ct = best_model.named_steps['prep']
    feat_names = get_feature_names(ct, X_test.columns)

    importances = pi_result.importances_mean
    idx = np.argsort(importances)[::-1][:20]  # top 20

    fig, ax = plt.subplots()
    ax.bar(range(len(idx)), importances[idx])
    ax.set_xticks(range(len(idx)))
    ax.set_xticklabels([feat_names[i] if i < len(feat_names) else f'f{i}' for i in idx], rotation=90)
    ax.set_title('Importancia por permutación (Top 20)')
    plt.tight_layout()
    plt.show()
except Exception as e:
    print('[WARN] Falló la importancia por permutación:', e)

# 4.2 PDP para 3 features numéricas más relevantes (si existen)
num_candidates = [c for c in X_test.columns if pd.api.types.is_numeric_dtype(X_test[c])]
top_for_pdp = num_candidates[:3]

if top_for_pdp:
    try:
        for feat in top_for_pdp:
            fig, ax = plt.subplots()
            PartialDependenceDisplay.from_estimator(best_model, X_test, [feat], ax=ax)
            ax.set_title(f'PDP — {feat}')
            plt.show()
    except Exception as e:
        print('[WARN] PDP no disponible:', e)
else:
    print('[INFO] Sin features numéricos candidatos para PDP.')


In [ ]:
# == 5. Exportación de artefactos
PIPE_PATH = ARTIF_DIR / 'preprocess_pipeline.joblib'
MODEL_PATH = ARTIF_DIR / 'model.joblib'
META_PATH  = ARTIF_DIR / 'metadata.json'

joblib.dump(best_model.named_steps['prep'], PIPE_PATH)
# Si el modelo es Pipeline('prep' + 'clf'), guardamos el pipeline completo para inferencia
joblib.dump(best_model, MODEL_PATH)

meta = {
    'timestamp': pd.Timestamp.now().isoformat(),
    'model': best_name,
    'metrics': {'roc_auc': float(res_rf['roc_auc'] if best_name=='RandomForest' else res_base['roc_auc']),
                'pr_auc': float(res_rf['pr_auc'] if best_name=='RandomForest' else res_base['pr_auc'])},
    'feature_columns': X.columns.tolist(),
    'target': 'riesgo_corrupcion',
    'notes': 'Reemplace datos sintéticos por sus fuentes reales.'
}
with open(META_PATH, 'w', encoding='utf-8') as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

PIPE_PATH, MODEL_PATH, META_PATH


In [ ]:
# == 6. Inferencia (función)
def predict_from_dataframe(df_new: pd.DataFrame, model_path='artifacts/model.joblib') -> pd.DataFrame:
    model = joblib.load(model_path)
    # Asegurar columnas esperadas (rellenar faltantes)
    expected = model.named_steps['prep'].get_feature_names_out if hasattr(model.named_steps['prep'], 'get_feature_names_out') else None
    # Aplicaremos el pipeline del modelo que maneja columnas + OHE + scaler internamente
    prob = model.predict_proba(df_new)[:, 1]
    pred = (prob >= 0.5).astype(int)
    out = df_new.copy()
    out['prob_riesgo'] = prob
    out['pred_riesgo'] = pred
    return out

# Ejemplo mínimo con 5 filas de df (tomado del test)
sample = X_test.head(5).copy()
predict_from_dataframe(sample).head(5)


In [ ]:
# == 7. Guardar script de inferencia por archivo (CSV)
script_text = '''#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Uso:
    python scripts/predict_file.py --input path/to/obras_nuevas.csv --output predicciones.csv

Requiere:
    - artifacts/model.joblib (pipeline + modelo)
"""
import argparse
import pandas as pd
import joblib

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--input', required=True, help='Ruta del CSV de entrada')
    parser.add_argument('--output', required=True, help='Ruta del CSV de salida')
    parser.add_argument('--model', default='artifacts/model.joblib', help='Ruta al modelo .joblib')
    args = parser.parse_args()

    df = pd.read_csv(args.input)
    model = joblib.load(args.model)

    prob = model.predict_proba(df)[:, 1]
    pred = (prob >= 0.5).astype(int)

    df_out = df.copy()
    df_out['prob_riesgo'] = prob
    df_out['pred_riesgo'] = pred

    df_out.to_csv(args.output, index=False)
    print(f'[OK] Predicciones guardadas en: {args.output}')

if __name__ == '__main__':
    main()
'''
(SCRIPT_DIR / 'predict_file.py').write_text(script_text, encoding='utf-8')
print('[OK] scripts/predict_file.py creado.')
